# BlueBERT

In [2]:
# Load model directly
from transformers import TrainingArguments, Trainer, AutoTokenizer, DataCollatorForLanguageModeling, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import load_dataset
import pandas as pd
import torch

model = AutoModelForSequenceClassification.from_pretrained("bionlp/bluebert_pubmed_uncased_L-12_H-768_A-12", num_labels=3)
print(model)

# Move model to the available device (cuda or cpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
model.to(device)

/root/programming-assignments/bioinfo/cap5510/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bionlp/bluebert_pubmed_uncased_L-12_H-768_A-12 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [3]:
def parse_context(context):
    result = ""
    # Combine all of the sentences of a context into one string
    for sentence in context['contexts']:
        result += sentence + " "
    
    return result

def parse_all_contexts(context_list):
    end = []
    # Go through all of the contexts in a dataset
    for context in context_list:
        result = parse_context(context)
        end.append(result)
    
    return end

def preprocess(data):
    questions = data['question']
    context = parse_all_contexts(data['context'])
    final_decision = data['final_decision']

    # Combine the question, context, and long answer
    text = [f"Context: {c} Question: {q}" for c, q in zip(context, questions)]
    token_input = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512)

    # Labels
    label_mapping = {"yes": 0, "no": 1, "maybe": 2}
    labels = [label_mapping[l] for l in final_decision]

    # Debugging information
    print(f"Processed text: {text}", len(text))
    print(f"Labels: {labels}", len(labels))
    print(f"Tokenized input: {token_input['input_ids']}", len(token_input['input_ids']))

    # Add the labels to the token_input
    token_input['labels'] = torch.tensor(labels, dtype=torch.long)

    return token_input

def predict_answer(context, question):
    text = f"Context: {context} Question: {question}"
    token_input = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512).to(device)

    # Generate model output (logits)
    with torch.no_grad():
        outputs = model(**token_input)

    # Extract the logits from the model output
    logits = outputs.logits

    # Get the predicted class (use argmax to get the index of the highest score)
    predicted_class = torch.argmax(logits, dim=-1).item()

    # Map the index to the label
    label_mapping = {0: "yes", 1: "no", 2: "maybe"}
    prediction = label_mapping[predicted_class]

    return prediction

In [ ]:
# Parameters
dataset_size = 300
train_size = 200
validate_size = 50
test_size = 50
num_epochs = 5
learning_rate = 2e-5
weight_decay = 0.01

# Get the dataset
dataset = load_dataset("qiaojin/PubMedQA", "pqa_labeled")
small_dataset = dataset['train'].select(range(dataset_size))

# Tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained("bionlp/bluebert_pubmed_uncased_L-12_H-768_A-12")
tokenized_input = small_dataset.map(preprocess, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Split the dataset
train_dataset, validate_dataset, test_dataset = torch.utils.data.random_split(tokenized_input, [train_size, validate_size, test_size])

# Train the model
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=num_epochs,
    eval_strategy="epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validate_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

# Evaluate the model
results = trainer.evaluate(test_dataset)
print("Test results", "\n\n", results, "\n\n")

Epoch,Training Loss,Validation Loss
1,No log,2.324460
2,No log,2.105311
3,1.609600,1.566154
4,1.609600,2.001193
5,1.158800,1.760421


Test results 

 {'eval_loss': 1.5797337293624878, 'eval_runtime': 1.4521, 'eval_samples_per_second': 34.433, 'eval_steps_per_second': 34.433, 'epoch': 5.0} 


Context: Treatment of neonatal testicular torsion has two objectives: salvage of the involved testicle (which is rarely achieved) and preservation of the contralateral gonad. The second goal universally involves contralateral testicular scrotal fixation to prevent the future occurrence of contralateral torsion. However, there is controversy with regards to management of a synchronous contralateral hydrocele. It has been our policy not to address the contralateral hydrocele through an inguinal incision to minimize potential injury to the spermatic cord. Our objective in this study was to determine whether the decision to manage a contralateral hydrocele in cases of neonatal testicular torsion solely through a scrotal approach is safe and effective. We reviewed all cases of neonatal testicular torsion occurring at our institution b

In [6]:
count = 0

# Test the model
for i in range(len(test_dataset)):
    context = parse_context(test_dataset[i]['context'])
    question = test_dataset[i]['question']
    answer = test_dataset[i]['final_decision']
    prediction = predict_answer(context, question)

    print(f"Context: {context}")
    print()
    print(f"Question: {question}")
    print()
    print(f"Predicted answer: {prediction}")
    print()
    print(f"Actual answer: {answer}")
    print("="*120)

    if prediction == answer:
        count += 1

print(f"Accuracy: {count/len(test_dataset)}")

Context: Treatment of neonatal testicular torsion has two objectives: salvage of the involved testicle (which is rarely achieved) and preservation of the contralateral gonad. The second goal universally involves contralateral testicular scrotal fixation to prevent the future occurrence of contralateral torsion. However, there is controversy with regards to management of a synchronous contralateral hydrocele. It has been our policy not to address the contralateral hydrocele through an inguinal incision to minimize potential injury to the spermatic cord. Our objective in this study was to determine whether the decision to manage a contralateral hydrocele in cases of neonatal testicular torsion solely through a scrotal approach is safe and effective. We reviewed all cases of neonatal testicular torsion occurring at our institution between the years 1999 and 2006. Age at presentation, physical examination, ultrasonographic and intraoperative findings were recorded. Patients were followed a